In [6]:
# Load Libraries
import warnings
from datetime import datetime

import tensorflow as tf
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional
from keras.layers import Embedding, LSTM, Input
from keras.layers.core import Dense, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from model_evaluator import Evaluator
from model_preprocessor import Preprocessor

warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

In [10]:
def bilstm(max_len=73, emb_dim=32, max_vocab_len=40, W_reg=regularizers.l2(1e-4)):
    """BiLSTM model with the Keras Sequential model"""
    input = Input(shape=(max_len,), dtype="int32", name="bilstm_input")
    emb = Embedding(input_dim=max_vocab_len, output_dim=emb_dim, input_length=max_len, W_regularizer=W_reg)(input)
    emb = Dropout(0.5)(emb)
    bilstm = Bidirectional(LSTM(128))(emb)
    bilstm = Dropout(0.5)(bilstm)
    output = Dense(21, activation='softmax', name='bilstm_output')(bilstm)
    model = Model(input=[input], output=[output])
    return model

In [8]:
# Load data using model preprocessor
x_train, x_test, y_train, y_test = Preprocessor.load_data()

In [11]:
# Define Deep Learning Model
model_name = "BILSTM"
model = bilstm()

''' Training phrase '''
epochs = 5
batch_size = 64
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy(),
                       Evaluator.precision, Evaluator.recall, Evaluator.fmeasure])

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.11)

Instructions for updating:
Use tf.cast instead.
Train on 1149965 samples, validate on 142131 samples
Epoch 1/5
1149965/1149965 [==============================] - 1326s 1ms/step - loss: 0.7342 - accuracy: 0.7832 - categorical_accuracy: 0.7153 - precision: 0.8765 - recall: 0.7150 - fmeasure: 0.7846 - val_loss: 0.4807 - val_accuracy: 0.8546 - val_categorical_accuracy: 0.7873 - val_precision: 0.9211 - val_recall: 0.8136 - val_fmeasure: 0.8636
Epoch 2/5
1149965/1149965 [==============================] - 1346s 1ms/step - loss: 0.4803 - accuracy: 0.8588 - categorical_accuracy: 0.8083 - precision: 0.9183 - recall: 0.8167 - fmeasure: 0.8640 - val_loss: 0.3513 - val_accuracy: 0.8969 - val_categorical_accuracy: 0.8250 - val_precision: 0.9379 - val_recall: 0.8666 - val_fmeasure: 0.9005
Epoch 3/5
1149965/1149965 [==============================] - 1350s 1ms/step - loss: 0.3832 - accuracy: 0.8895 - categorical_accuracy: 0.8371 - precision: 0.9334 - recall: 0.8550 - fmeasure: 0.8921 - val_loss: 0.2828

In [ ]:
model_json = model.to_json()
with open("./saved_model/bilstm.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("./saved_model/bilstm.h5")